# Definitions to calculate coupling efficiency

The calculation of the coupling efficiency shown in this script is based on the information found [here](https://is.gd/XRjN6o). Modifications were made to the geometrical efficiency factor to allow calculations of the coupling efficiency when the laser diode and the waveguide are separated by some distance.

In order to use this package (the definitions shown in this script) you have to place the file "efficiency.py" in the same folder of the script that is going to use the definitions and functions defined here. In addition, at the beggining of your script, this this package has to be imported with the following command

```python
import efficiency as ef
```

Once this is done, the classes and methods from this package can be access preceded by the prefix ef.

For example to create an instance of a square waveguide we use:
```python
waveguide1 = ef.SquaredWaveguide(n_core, n_clad, core_thickness, clad_thickness)
```


## 1. Import all the necessary packages 

The following python packages should be installed (dependencies):
- numpy

Once this packages are installed, we import them into our script with the following commands

In [ ]:
import numpy as np

## 2. Define the SquareWaveguide class

This class allows to have a general definition of a waveguide with a squared cross-section, which depends on some inital parameters, like the indices of refractions of the core and cladding, and to automaticatically derive some values (e.g. the acceptance angle) once the initial parameters are introduced.

The inital parameters/attributes of the waveguide are:

**n_core **: _index of refraction of the waveguide's core._

**n_clad**: _index of refraction of the waveguide's cladding._

**core_thickness**: _thickness of the core layer. Units: um_

**clad_thickness**: _thickness of the cladding layer. Units: um_


Later, we can create an instance of a a waveguide with a square cross-section and store it in the variable waveguide1 with

```python
waveguide1 = ef.SquaredWaveguide(n_core, n_clad, core_thickness, clad_thickness)
```


The function/method "get_acceptance_angle()" is defined inside the class to calculate the acceptance angle.

### Acceptance angle

To calculate the acceptance angle, first the numerical apperture NA is calculated as:

In [3]:
%%latex 
$$NA = \sqrt{n_{core}^2-n_{cladd.}^2}$$

<IPython.core.display.Latex object>

Once that we have the NA, the acceptance angle can be calculated as:

In [13]:
%%latex 
$$\theta_{0,max} = \arcsin({NA})$$

<IPython.core.display.Latex object>

In [ ]:
class SquaredWaveguide(object):
    """
    A waveguide with a squared cross section.

    class input attributes
    ----------------------
    n_core: index of refraction of the waveguide's core.
    n_clad: index of refraction of the waveguide's cladding.
    core_thickness: thickness of the core layer. Units: um
    clad_thickness: thickness of the cladding layer. Units: um
    """
    def __init__(self, n_core, n_clad, core_thickness, clad_thickness):
        # TODO - check for validity and units.
        # basic parameters
        self.n_core = n_core
        self.n_clad = n_clad
        self.core_thickness = core_thickness
        self.clad_thickness = clad_thickness
        # calculate additional parameters
        self.theta = self.get_acceptance_angle()  # acceptance angle

    def get_acceptance_angle(self):
        # calculate numerical aperture
        na = np.sqrt(self.n_core**2 - self.n_clad**2)
        # derive acceptance angle from numerical aperture
        theta = np.arcsin(na)  # theta is in rad
        return theta

## 3. Define the LaserDiode class

Similarly to the case of the waveguide, this allows to have a general definition of the laser diode and to automatically calculate some properties of the laser once the intial attributes of the laser are defined.

The inital parameters/attributes of the laser diode are:

**lda**: _wavelength_.

**fwhm_slow**: _Full width at half maximum on the slow axis. Units: deg_

**fwhm_fast**: _Full width at half maximum on the fast axis. Units: deg_


We create an instance of a laser diode which is stored in the variable laser1 with

```python
laser1 = ef.LaserDiode(lda, fwhm_slow, fwhm_fast)
```

The functions/methods "get_power_distribution_coefficient()" and "get_gaussian_beam_parameters()"
are defined inside the class to calculate the power distribution coefficients and the Gaussian beam parameter respectively.

In addition, the function "calculate_beam_width(x)" is defined to calculate the beam width at a distance x from the source.

### Power Distribution Coefficient 
The power distribution coefficients are used to describe the power angular distribution (radiance, B) of the light emitted by the laser diode.

The source is assumed to have sinusoidal power angular intensity distribution (circularly symmetric), which is given by the expression:

In [24]:
%%latex 
$$B(\theta) = B_0 (\cos{\theta})^m$$

<IPython.core.display.Latex object>

The coefficient _**m**_ can be calculated from the FMHM as follows: 

In [10]:
%%latex 
$$m = \frac{\log(0.5)}{\log(\cos(\frac{FWHM}{2}))}$$

<IPython.core.display.Latex object>

### Gaussian beam parameters
The Gaussian beam parameter are calculated as follows:

Half divergence angle:

In [19]:
%%latex 
$$  \theta = \frac{\sqrt{2} \cdot FWHM}{2 \cdot \log(2)} $$

<IPython.core.display.Latex object>

Minimum half width:

In [21]:
%%latex 
$$ w_0 = \frac{\lambda}{\pi \cdot \theta} $$

<IPython.core.display.Latex object>

Rayleight range:

In [23]:
%%latex
$$    x_0 = \frac{\pi \cdot w_0^2}{\lambda} $$

<IPython.core.display.Latex object>

### Calculate the beam width at a distance x from the source

Once we have the Gaussian beam parameters, we can calculate the beam half width at a distance x from the source. This is done by using the expression:

In [4]:
%%latex
$$w(x) = w_0 \cdot \sqrt{1 + \left(\frac{x}{x_o}\right)^2}$$

<IPython.core.display.Latex object>

In [1]:
class LaserDiode(object):
    """
    A laser diode object.

    class input attributes (Properties of the laser diode)
    ----------------------
    lda: wavelength. Units: nm
    fwhm_slow: Full width at half maximum on the slow axis. Units: deg
    fwhm_fast: Full width at half maximum on the fast axis. Units: deg
    width: width of the emitting surface. Units: um
    height: height of the emitting surface. Units: um
    """

    def __init__(self, lda, fwhm_slow, fwhm_fast):
        # TODO - check for validity and units.
        self.lda = lda
        self.FWHM_slow = fwhm_slow
        self.FWHM_fast = fwhm_fast
        # self.width = width
        # self.height = height
        # # calculate the emitting surface
        # self.As = self.width * self.height

        # calculate power distribution coefficients
        self.l_coefficient = self.get_power_distribution_coefficient(self.FWHM_slow)
        self.t_coefficient = self.get_power_distribution_coefficient(self.FWHM_fast)
        # calculate gaussian beam parameters: half divergence angle, minimum half width and Rayleigh range
        self.theta_slow, self.wo_slow, self.xo_slow = self.get_gaussian_beam_parameters(self.FWHM_slow)
        self.theta_fast, self.wo_fast, self.xo_fast = self.get_gaussian_beam_parameters(self.FWHM_fast)

    def get_power_distribution_coefficient(self, fwhm):
        """
        Calculate power distribution coefficient. This coefficient is later used in the expression of the radiance
        of laser diode. (Can be used for both the parallel and perpendicular coefficients). This calculation assumes
        that the radiance distribution is circularly symmetric.

        :return:
        m: power distribution coefficient.
        """
        # change the unit of the fwhm angle from deg to rad
        fwhm_rad = np.deg2rad(fwhm)
        # calculate the coefficient
        m = np.log(0.5)/np.log(np.cos(fwhm_rad/2))
        # take the integer part
        m = np.floor(m)
        return m

    def get_gaussian_beam_parameters(self, fwhm):
        """
        Calculate the half divergence angle, the beam minimum half width and the Rayleigh range from the FWHM.

        :param:
        fwhm: Full width at half maximum
        :return:
        theta: half divergence angle of the gaussian beam in radians.
        wo: beam minimum half width.
        xo: Rayleigh range.
        """
        # change the unit of the fwhm angle from deg to rad
        fwhm_rad = np.deg2rad(fwhm)
        # calculate half divergence angle
        theta = (np.sqrt(2) * fwhm_rad) / (2 * np.sqrt(np.log(2)))
        # change lambda from nm to um
        lda_um = self.lda * 1e-3
        # derive the other two parameters
        wo = lda_um / (np.pi * theta)
        xo = np.pi * wo ** 2 / lda_um
        return theta, wo, xo

    def calculate_beam_width(self, x):
        """
        Calculate the beam half width at a distance x from the origin. The half width is calculated for the slow and
        fast axis, w_slow and w_fast.

        :param:
        x: distance, with respect to the origin, when the beam half width is calculated
        :return:
        w_slow: beam half width at a distance x from the source for the slow axis
        w_fast: beam half width at a distance x from the source for the fast axis
        """
        w_slow = self.wo_slow * np.sqrt(1 + (x / self.xo_slow) ** 2)
        w_fast = self.wo_fast * np.sqrt(1 + (x / self.xo_fast) ** 2)
        return w_slow, w_fast

## 4. Define the Calculator class

In this class we bring together the information of the laser diode and the waveguide, so that the coupling efficiency can be calculated.

The efficiency of direct coupling of a source into an optical fiber is affected by three different factors.
Theses factors are: geometrical losses, Fresnel losses and angular losses.

In [6]:
%%latex
$$ \eta = \frac{P_{input}}{P_{source}} = \eta_{geo} \cdot \eta_{Fresnel} \cdot \eta_{ang} $$

<IPython.core.display.Latex object>

where Pinput is the power coupled into the waveguide

The functions/methods "geometrical_losses(x)" and "fresnel_losses()" and "angular_losses()"
are defined inside the class to calculate the geometrical, fresnel and angular efficiency factors respectively.

In addition, the "total_efficiency()" method calls the previous three functions and multiplies them to get the total efficiency.

### 1. Geometrical efficiency factor

The light emmited by a light source will coupled into the waveguide depending on the relative spot size of the light source and the core area of the waveguide. 

If the laser diode has different divergence angles on each direction, the spot size will have an elliptic shape. Thus, the spot size area at a distance x from the light source can be calculated with the following formula:

In [3]:
%%latex
$$ A_{spotsize}(x) = \pi \cdot w_{fast}(x) \cdot w_{slow}(x) $$

<IPython.core.display.Latex object>

where w_fast and w_slow is the beam half width at that distance x calculated for the fast and slow axis.

The area of the waveguide is given by:

In [4]:
%%latex
$$ A_{core} = t ^ 2 $$

<IPython.core.display.Latex object>

where t is the thickness of the waveguide's core.

The light incident outside the core area will not be guided by the waveguide. Therefore, there are three possible configurations for emmiter and waveguide are:

a. If the source dimensions are smaller than the dimension of the waveguide core area, all the light will be coupled into the fiber.

In [10]:
%%latex
$$ if \quad t \geq w_{fast}(x) \quad and \quad t \geq w_{slow}(x)  $$
$$ then \quad \eta_{geo} = 1 $$

<IPython.core.display.Latex object>

b. If the source is larger than the waveguide core area, the geometrical factor for coupling efficiency is given by the ratio of light captured by the fiber, i.e. the ratio of the areas.

In [12]:
%%latex
$$ if \quad t < w_{fast}(x) \quad and \quad t < w_{slow}(x)  $$
$$ then \quad \eta_{geo} = \frac{A_{core}}{A_{spotsize}} $$

<IPython.core.display.Latex object>

c. If the waveguide core area is larger than one side of the source, but smaller than the second side. The geometrical factor for coupling efficiency is given by the following ratio

In [17]:
%%latex
$$ if \quad t \geq w_{fast}(x) \quad and  \quad t < w_{slow}(x) $$
$$ then \quad \eta_{geo} = \frac{t}{w_{slow}(x)} $$

<IPython.core.display.Latex object>

In [19]:
%%latex
$$ if \quad t < w_{fast}(x) \quad and  \quad t \geq w_{slow}(x) $$
$$ then \quad \eta_{geo} = \frac{t}{w_{fast}(x)} $$

<IPython.core.display.Latex object>

### 2. Frenesel efficiency factor

The light incident on an interface between two dielectric media of different refractive index is partially transmitted and partially reflected (assuming linear, homogeneous and isotropic media).  The proportion of light reflected and transmitteddepend on light polarisation and incidence angle and can be calculated from Fresnel equations based on electromagnetic theory.

The power reflectance at normal incidence is then given by:

In [20]:
%%latex
$$ R = \frac{(n_{core}-1)^2}{(n_{core}+1)^2} $$

<IPython.core.display.Latex object>

and the Fresnel factor for coupling efficiency is approximatly equal to:

In [21]:
%%latex
$$ \eta_{Fresnel} = 1 - R $$

<IPython.core.display.Latex object>

### 3. Angular efficiency factor

Once that the source angular distribution and the fiber acceptance angle are well defined (see definition of the LaserDiode class), it can be demonstrated that the angular coupling efficiency factor is 

In [18]:
%%latex
$$\eta_{angular} = 1 - (\cos(\theta))^{(m + 1)}$$

<IPython.core.display.Latex object>

In the case of coupling a laser diode to a waveguide, the emitter is not circularly symmetric and has different divergence angle depending of azimut angle. In fact, laser diodes typically have two main directions.  One direction with smaller dimension and higher divergence called fast axis, and another direction with larger dimension and lower divergence called slow axis. 

However, we can get a good approximate for coupling efficiency by calculating efficiency in both axis separatly, by supposing a revolution symmetry for both axis separatly, and multiply the square root of the computed efficiency for each axis.

Therefore we will calculate the power distribution coefficients, m, for the slow and fast axis separatedly. The letter **L** will be used to denote the power distribution coefficient of the slow axis, and the letter **T** will be used for the fast axis.

Once we have the power distribution coefficients **L** and **T**,

In [ ]:
class Calculator(object):
    """
    Calculate coupling efficiency between a laser diode and a squared waveguide.

    class input attributes
    ----------------------
    waveguide: a SquareWaveguide object
    laserdiode: a LaserDiode object
    """

    def __init__(self, waveguide, laserdiode):
        # check that the inputs are instances of the SquaredWaveguide and LaserDiode classes
        if not isinstance(waveguide, SquaredWaveguide):
            raise TypeError("waveguide input must be a SquaredWaveguide object.")
        if not isinstance(laserdiode, LaserDiode):
            raise TypeError("laserdiode input must be a LaserDiode object.")

        self.wv = waveguide
        self.ld = laserdiode

    def geometrical_losses(self, x=0):
        """
        Calculate the geometrical coupling efficiency factor when the waveguide is separated from the laser diode
        by a distance of x.

        :param:
        x: distance of separation between the laser diode and the waveguide
        :return:
        n_geom: geometrical factor for coupling efficiency
        """

        # give shorter names to the useful variables
        wv_l = self.wv.core_thickness
        # calculate the area of the spot size at a distance x from the light source
        ld_w, ld_h = self.ld.calculate_beam_width(x)
        # Note that the beam has an elliptical shape. Therefore, the area is given by pi*a*b,
        # where a and b are the ellipse semi axis.
        ld_A = np.pi * ld_h * ld_w

        # initialize the value of n_geom
        n_geom = 0

        if wv_l >= ld_w and wv_l >= ld_h:
            # if the source dimensions are smaller than the dimension of the waveguide core area,
            # all the light will be coupled into the fiber.
            n_geom = 1

        elif wv_l < ld_w and wv_l < ld_h:
            # if the source is larger than the waveguide core area, the geometrical factor for coupling efficiency
            # is given by the ratio of light captured by the fiber, i.e. the ratio of the areas.
            n_geom = (wv_l ** 2) / ld_A

        elif wv_l < ld_w and wv_l >= ld_h:
            # if the waveguide core area is larger than one side of the source, but smaller than the second side.
            # The geometrical factor for coupling efficiency is given by the following ratio
            n_geom = wv_l / ld_w

        elif wv_l >= ld_w and wv_l < ld_h:
            # if the waveguide core area is larger than one side of the source, but smaller than the second side.
            # The geometrical factor for coupling efficiency is given by the following ratio
            n_geom = wv_l / ld_h

        return n_geom

    def fresnel_losses(self):
        """
        Calculate Fresnel coupling efficiency factor.

        :return:
        n_fresnel: Fresnel factor for coupling efficiency
        """
        # give shorter names to the useful variables
        n_core = self.wv.n_core
        # calculate power reflectance
        r = (n_core - 1)**2 / (n_core + 1)**2
        n_fresnel = 1 - r
        return n_fresnel

    def angular_losses(self):
        """
        Calculate angular coupling efficiency factor
        (due to the spatial distribution of the power emitted by the source).

        :return:
        """
        # get relevant parameters
        theta = self.wv.theta
        k = self.ld.l_coefficient
        t = self.ld.t_coefficient

        # calculate the angular coupling for each perpendicular direction separately (assuming circular symmetry)
        n_angular_l = 1 - (np.cos(theta)) ** (k + 1)
        n_angular_t = 1 - (np.cos(theta)) ** (t + 1)
        # approximate the total angular efficiency factor by multiplying the square roots of each individual factor.
        n_angular = np.sqrt(n_angular_l * n_angular_t)
        return n_angular

    def total_efficiency(self, x):
        """
        Calculate the total efficiency for a separation between the laser diode and the waveguide of x.
        Multiplies the results of the geometrical, fresnel and angular efficiency factors.

        :param:
        x: distance of separation between the laser diode and the waveguide
        :return:
        n_total: coupling efficiency
        """
        n_geom = self.geometrical_losses(x)
        n_fresnel = self.fresnel_losses()
        n_angular = self.angular_losses()
        n_total = n_geom * n_fresnel * n_angular
        return n_total